In [5]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

In [6]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [7]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [8]:
analize = Analizer(0.1)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt,Architecture
0,model_14_7_6,0.998567,0.994798,0.999067,0.997611,0.172856,0.462961,0.176691,0.328245,"Hidden Size=[35], regularizer=0.1, learning_ra..."
1,model_14_7_5,0.998562,0.995669,0.999219,0.998009,0.173452,0.385403,0.147853,0.273614,"Hidden Size=[35], regularizer=0.1, learning_ra..."
2,model_14_7_7,0.998546,0.993937,0.998916,0.997218,0.175324,0.539590,0.205174,0.382218,"Hidden Size=[35], regularizer=0.1, learning_ra..."
3,model_14_7_4,0.998522,0.996532,0.999370,0.998402,0.178254,0.308660,0.119291,0.219546,"Hidden Size=[35], regularizer=0.1, learning_ra..."
4,model_14_7_8,0.998508,0.993100,0.998770,0.996837,0.179975,0.614054,0.232852,0.434665,"Hidden Size=[35], regularizer=0.1, learning_ra..."
...,...,...,...,...,...,...,...,...,...,...
442,model_18_8_0,0.986129,0.975117,0.989893,0.986089,1.672893,1.611881,2.086072,1.835030,"Hidden Size=[45], regularizer=0.1, learning_ra..."
453,model_17_8_1,0.985611,0.979866,0.998658,0.982431,1.735261,4.158459,0.047088,2.223697,"Hidden Size=[45], regularizer=0.01, learning_r..."
459,model_12_7_5,0.985205,0.989045,0.976057,0.983359,1.784249,1.269617,2.442341,1.821487,"Hidden Size=[30], regularizer=0.1, learning_ra..."
462,model_6_9_4,0.985109,0.987256,0.972666,0.984825,1.795853,1.221799,1.245121,1.232775,"Hidden Size=[15], regularizer=0.1, learning_ra..."
